# **Stellar Dataset**

### **Data Description:**
In astronomy, stellar classification is the classification of stars based on their spectral characteristics. The classification scheme of galaxies, quasars, and stars is one of the most fundamental in astronomy. The early cataloguing of stars and their distribution in the sky has led to the understanding that they make up our own galaxy and, following the distinction that Andromeda was a separate galaxy to our own, numerous galaxies began to be surveyed as more powerful telescopes were built. This datasat aims to classificate stars, galaxies, and quasars based on their spectral characteristics.


### **Data Content:**
The data consists of observations of space taken by the SDSS (Sloan Digital Sky Survey). Every observation is described by 17 feature columns and 1 class column which identifies it to be either a star, galaxy or quasar.

### **Data Dictionary:**
- obj_ID = Object Identifier, the unique value that identifies the object in the image catalog used by the CAS
- alpha = Right Ascension angle (at J2000 epoch)
- delta = Declination angle (at J2000 epoch)
- u = Ultraviolet filter in the photometric system
- g = Green filter in the photometric system
- r = Red filter in the photometric system
- i = Near Infrared filter in the photometric system
- z = Infrared filter in the photometric system
- run_ID = Run Number used to identify the specific scan
- rereun_ID = Rerun Number to specify how the image was processed
- cam_col = Camera column to identify the scanline within the run
- field_ID = Field number to identify each field
- spec_obj_ID = Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class)
- class = object class (galaxy, star or quasar object) << your target
- redshift = redshift value based on the increase in wavelength
- plate = plate ID, identifies each plate in SDSS
- MJD = Modified Julian Date, used to indicate when a given piece of SDSS - - - data was taken
- fiber_ID = fiber ID that identifies the fiber that pointed the light at the focal plane in each observation

#### Import libraries and read files

In [28]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import ggplot, aes, geom_density, geom_line, geom_point, ggtitle

# Modeling process
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV


In [29]:
df = pd.read_csv("data/stellar.csv")
df.head()

Unnamed: 0        obj_ID       alpha      delta         u         g  \
0       41029  1.237649e+18  194.748212  -0.911226  25.77469  22.72579   
1       42888  1.237661e+18  140.525977  35.614836  21.94718  21.31617   
2       82610  1.237658e+18  125.922894  38.044046  23.47268  21.34390   
3       89586  1.237664e+18   18.634831   0.468756  20.03793  18.13051   
4       14627  1.237666e+18   52.832458   1.215699  20.72916  20.34843   

          r         i         z  run_ID  rerun_ID  cam_col  field_ID  \
0  20.84263  19.80384  19.29726     756       301        1       527   
1  20.21319  19.45814  19.09832    3560       301        4       221   
2  19.41544  18.67742  18.14655    2822       301        2       135   
3  17.21534  16.80004  16.48915    4263       301        5       240   
4  20.11169  19.75053  19.74247    4849       301        6       807   

    spec_obj_ID   class  redshift  plate    MJD  fiber_ID  
0  4.271919e+18  GALAXY  0.525770   3794  55241       926  
1  5.228710e+18  GALAXY  0.439029   4644  55922       111  
2  4.233595e+18  GALAXY  0.414493   3760  55268       770  
3  1.217236e+18  GALAXY  0.091736   1081  52531       503  
4  8.028670e+17     QSO  1.562706    713  52178       365

#### Data Exploring:

In [30]:
# check data structure
df.shape

(80000, 19)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   80000 non-null  int64  
 1   obj_ID       80000 non-null  float64
 2   alpha        80000 non-null  float64
 3   delta        80000 non-null  float64
 4   u            80000 non-null  float64
 5   g            80000 non-null  float64
 6   r            80000 non-null  float64
 7   i            80000 non-null  float64
 8   z            80000 non-null  float64
 9   run_ID       80000 non-null  int64  
 10  rerun_ID     80000 non-null  int64  
 11  cam_col      80000 non-null  int64  
 12  field_ID     80000 non-null  int64  
 13  spec_obj_ID  80000 non-null  float64
 14  class        80000 non-null  object 
 15  redshift     80000 non-null  float64
 16  plate        80000 non-null  int64  
 17  MJD          80000 non-null  int64  
 18  fiber_ID     80000 non-null  int64  
dtypes: f

In [32]:
# check for duplicates
df.duplicated().sum()

0

In [33]:
# check for missing values
df.isnull().sum().sum()

0

In [34]:
# drop unneccery columns
df.drop('Unnamed: 0', axis='columns', inplace=True)

In [35]:
# change ID's columns dtype 
df['fiber_ID'] = df['fiber_ID'].astype(str)
df['spec_obj_ID'] = df['spec_obj_ID'].astype(str)
df['field_ID'] = df['field_ID'].astype(str)
df['rerun_ID'] = df['rerun_ID'].astype(str)
df['run_ID'] = df['run_ID'].astype(str)
df['obj_ID'] = df['obj_ID'].astype(str)
df['plate'] = df['plate'].astype(str)

In [36]:
df.dtypes

obj_ID          object
alpha          float64
delta          float64
u              float64
g              float64
r              float64
i              float64
z              float64
run_ID          object
rerun_ID        object
cam_col          int64
field_ID        object
spec_obj_ID     object
class           object
redshift       float64
plate           object
MJD              int64
fiber_ID        object
dtype: object

In [37]:
df.columns

Index(['obj_ID', 'alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'class', 'redshift',
       'plate', 'MJD', 'fiber_ID'],
      dtype='object')

#### Handling Multicollinearity

In [38]:
#check the corrolation
df.corr()

alpha     delta         u         g         r         i  \
alpha     1.000000  0.139960 -0.001654 -0.002469 -0.022075 -0.023910   
delta     0.139960  1.000000  0.002714  0.003994 -0.007344 -0.004691   
u        -0.001654  0.002714  1.000000  0.999450  0.049075  0.041356   
g        -0.002469  0.003994  0.999450  1.000000  0.056371  0.050771   
r        -0.022075 -0.007344  0.049075  0.056371  1.000000  0.963159   
i        -0.023910 -0.004691  0.041356  0.050771  0.963159  1.000000   
z        -0.002993  0.004123  0.998474  0.999330  0.048657  0.050628   
cam_col   0.020881  0.034869  0.003862  0.003836  0.009218  0.007912   
redshift  0.000072  0.032344  0.013408  0.021127  0.433485  0.493205   
MJD       0.022503  0.106413  0.028870  0.036236  0.669826  0.671740   

                 z   cam_col  redshift       MJD  
alpha    -0.002993  0.020881  0.000072  0.022503  
delta     0.004123  0.034869  0.032344  0.106413  
u         0.998474  0.003862  0.013408  0.028870  
g         0.999330  0.003836  0.021127  0.036236  
r         0.048657  0.009218  0.433485  0.669826  
i         0.050628  0.007912  0.493205  0.671740  
z         1.000000  0.003715  0.027788  0.033732  
cam_col   0.003715  1.000000  0.000841 -0.006703  
redshift  0.027788  0.000841  1.000000  0.386759  
MJD       0.033732 -0.006703  0.386759  1.000000

In [39]:
corr_matrix=df.corr()

# Create a Chart for the 6 Factors Correlations
sns.heatmap(corr_matrix, annot = True, cmap="YlGnBu")
plt.title(('Correlations Between Factors'))
sns.set(rc={'figure.figsize':(15,10)})

In [40]:
#Muticollinearity exists between features - ('u' and 'g'), ('g', 'r', 'i'), ('z', 'g', 'r', 'i'), ('spec_obj_ID' and 'plate')
#I will remove features - 'g', 'r', 'i', 'spec_obj_ID'.
df.drop(["g", "r", "i", "spec_obj_ID"], axis=1, inplace=True)

### Dimension reduction

In [41]:
#Drop the ID Columns and save them into ID dataframe
ID=df[['obj_ID','run_ID','rerun_ID','field_ID','fiber_ID']]
df.drop(['obj_ID','run_ID','rerun_ID','field_ID','fiber_ID'], axis=1, inplace=True)
ID

obj_ID run_ID rerun_ID field_ID fiber_ID
0       1.237648720163439e+18    756      301      527      926
1       1.237660764842296e+18   3560      301      221      111
2      1.2376575940770534e+18   2822      301      135      770
3      1.2376637847424205e+18   4263      301      240      503
4       1.237666302167286e+18   4849      301      807      365
...                       ...    ...      ...      ...      ...
79995  1.2376543814464968e+18   2074      301      211      335
79996      1.237662963855e+18   4072      301       60        1
79997  1.2376648370029212e+18   4508      301      141      139
79998  1.2376651070417797e+18   4571      301       31      517
79999  1.2376682949855155e+18   5313      301       96      202

[80000 rows x 5 columns]

In [42]:
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(ID)
ID_new = pd.DataFrame(data = principalComponents, columns = ['ID'])
ID_new


ID
0      1.598441e+13
1      3.939732e+12
2      7.110498e+12
3      9.198324e+11
4     -1.597592e+12
...             ...
79995  1.032313e+13
79996  1.740720e+12
79997 -1.324281e+11
79998 -4.024670e+11
79999 -3.590411e+12

[80000 rows x 1 columns]

In [43]:
df['ID']=ID_new
df

alpha      delta         u         z  cam_col   class  redshift  \
0      194.748212  -0.911226  25.77469  19.29726        1  GALAXY  0.525770   
1      140.525977  35.614836  21.94718  19.09832        4  GALAXY  0.439029   
2      125.922894  38.044046  23.47268  18.14655        2  GALAXY  0.414493   
3       18.634831   0.468756  20.03793  16.48915        5  GALAXY  0.091736   
4       52.832458   1.215699  20.72916  19.74247        6     QSO  1.562706   
...           ...        ...       ...       ...      ...     ...       ...   
79995  156.946589  58.686923  22.37756  18.34169        2  GALAXY  0.538937   
79996  355.649333  44.738093  18.02005  16.35642        4    STAR -0.001132   
79997  132.990801  25.816729  25.69788  19.68414        5  GALAXY  0.765474   
79998  245.971013  19.218462  18.29851  17.81232        4     QSO  1.501470   
79999  173.620117  20.620310  21.45405  17.21519        6  GALAXY  0.323639   

       plate    MJD            ID  
0       3794  55241  1.598441e+13  
1       4644  55922  3.939732e+12  
2       3760  55268  7.110498e+12  
3       1081  52531  9.198324e+11  
4        713  52178 -1.597592e+12  
...      ...    ...           ...  
79995   7089  56661  1.032313e+13  
79996   1886  53237  1.740720e+12  
79997  11122  58437 -1.324281e+11  
79998   2970  54589 -4.024670e+11  
79999   2504  54179 -3.590411e+12  

[80000 rows x 10 columns]

In [44]:
df.corr()

alpha     delta         u         z   cam_col  redshift  \
alpha     1.000000  0.139960 -0.001654 -0.002993  0.020881  0.000072   
delta     0.139960  1.000000  0.002714  0.004123  0.034869  0.032344   
u        -0.001654  0.002714  1.000000  0.998474  0.003862  0.013408   
z        -0.002993  0.004123  0.998474  1.000000  0.003715  0.027788   
cam_col   0.020881  0.034869  0.003862  0.003715  1.000000  0.000841   
redshift  0.000072  0.032344  0.013408  0.027788  0.000841  1.000000   
MJD       0.022503  0.106413  0.028870  0.033732 -0.006703  0.386759   
ID        0.013633  0.301188 -0.014838 -0.013720  0.048166 -0.065964   

               MJD        ID  
alpha     0.022503  0.013633  
delta     0.106413  0.301188  
u         0.028870 -0.014838  
z         0.033732 -0.013720  
cam_col  -0.006703  0.048166  
redshift  0.386759 -0.065964  
MJD       1.000000 -0.262421  
ID       -0.262421  1.000000

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   alpha     80000 non-null  float64
 1   delta     80000 non-null  float64
 2   u         80000 non-null  float64
 3   z         80000 non-null  float64
 4   cam_col   80000 non-null  int64  
 5   class     80000 non-null  object 
 6   redshift  80000 non-null  float64
 7   plate     80000 non-null  object 
 8   MJD       80000 non-null  int64  
 9   ID        80000 non-null  float64
dtypes: float64(6), int64(2), object(2)
memory usage: 6.1+ MB


In [46]:
df['ID']= df['ID'].astype(str)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   alpha     80000 non-null  float64
 1   delta     80000 non-null  float64
 2   u         80000 non-null  float64
 3   z         80000 non-null  float64
 4   cam_col   80000 non-null  int64  
 5   class     80000 non-null  object 
 6   redshift  80000 non-null  float64
 7   plate     80000 non-null  object 
 8   MJD       80000 non-null  int64  
 9   ID        80000 non-null  object 
dtypes: float64(5), int64(2), object(3)
memory usage: 6.1+ MB


### Data splitting

#### Simple random sampling

In [48]:
# create train/test split
train, test = train_test_split(df, train_size=0.7, random_state=123)

# dimensions of training data
train.shape

(56000, 10)

In [49]:
df.columns

Index(['alpha', 'delta', 'u', 'z', 'cam_col', 'class', 'redshift', 'plate',
       'MJD', 'ID'],
      dtype='object')

In [50]:
## test 7:16 PM

#### Stratified sampling

In [51]:
y = df["class"]
train_strat, test_strat = train_test_split(df, train_size=0.3, random_state=123, 
                                           stratify=y)

In [52]:
# response distribution for raw data
df["class"].value_counts(normalize=True)

GALAXY    0.594450
STAR      0.215938
QSO       0.189612
Name: class, dtype: float64

In [53]:
train_strat["class"].value_counts(normalize=True)

GALAXY    0.594458
STAR      0.215917
QSO       0.189625
Name: class, dtype: float64

In [54]:
test_strat["class"].value_counts(normalize=True)

GALAXY    0.594446
STAR      0.215946
QSO       0.189607
Name: class, dtype: float64

#### Data imbalances